<a href="https://colab.research.google.com/github/Sakshivemuri/Sakshivemuri/blob/main/car_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [15]:
# Import libraries
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import joblib

# Create synthetic dataset
random.seed(42)
years = np.random.randint(2000, 2024, 200)
kms_driven = np.random.randint(1000, 200000, 200)
fuel_types = np.random.choice(['Petrol', 'Diesel', 'CNG', 'Electric'], 200)
seller_types = np.random.choice(['Dealer', 'Individual'], 200)
transmissions = np.random.choice(['Manual', 'Automatic'], 200)
owners = np.random.choice(['First', 'Second', 'Third', 'Fourth & Above'], 200)

# Assume basic pricing formula
base_price = 10 + (2024 - years) * -0.2 + (kms_driven / -10000) + np.random.normal(0, 1, 200)
base_price = np.clip(base_price, 1, None)

# Build DataFrame
df = pd.DataFrame({
    'Year': years,
    'Kms_Driven': kms_driven,
    'Fuel_Type': fuel_types,
    'Seller_Type': seller_types,
    'Transmission': transmissions,
    'Owner': owners,
    'Car_Price': base_price
})

# Save dataset to Drive
df.to_csv('/content/drive/MyDrive/second_hand_car_dataset.csv', index=False)

print("✅ Dataset created and saved successfully!")
df.head()


✅ Dataset created and saved successfully!


,Year,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,Car_Price
0,2001,91327,Electric,Individual,Automatic,Fourth & Above,1.0000
1,2000,132410,Electric,Dealer,Automatic,Second,1.0000
2,2022,144326,Petrol,Dealer,Manual,Fourth & Above,1.0000
3,2022,183932,Electric,Individual,Manual,Third,1.0000
4,2017,12344,Petrol,Individual,Manual,Fourth & Above,6.3964


In [16]:
# Load dataset
df = pd.read_csv('/content/drive/MyDrive/second_hand_car_dataset.csv')

# Separate features and target
X = df.drop('Car_Price', axis=1)
y = df['Car_Price']

# One-hot encoding
X_encoded = pd.get_dummies(X)

# Save feature columns
feature_columns = X_encoded.columns.tolist()
joblib.dump(feature_columns, '/content/drive/MyDrive/car_feature_columns.pkl')

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

# Save scaler
joblib.dump(scaler, '/content/drive/MyDrive/car_scaler.pkl')

print("✅ Preprocessing done. Features encoded and scaled.")


✅ Preprocessing done. Features encoded and scaled.


In [17]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Save model
joblib.dump(model, '/content/drive/MyDrive/linear_regression_car_model.pkl')

print("✅ Model trained and saved successfully!")


✅ Model trained and saved successfully!


In [19]:
!pip install ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.3 MB/s eta 0:00:00


In [20]:
import joblib
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Load model, scaler, feature columns
model = joblib.load('/content/drive/MyDrive/linear_regression_car_model.pkl')
scaler = joblib.load('/content/drive/MyDrive/car_scaler.pkl')
feature_columns = joblib.load('/content/drive/MyDrive/car_feature_columns.pkl')

# Create widgets for input
year_widget = widgets.IntSlider(value=2015, min=2000, max=2024, description='Year:')
kms_widget = widgets.IntText(value=50000, description='KMs Driven:')
fuel_widget = widgets.Dropdown(options=['Petrol', 'Diesel', 'CNG', 'Electric'], description='Fuel Type:')
seller_widget = widgets.Dropdown(options=['Dealer', 'Individual'], description='Seller Type:')
transmission_widget = widgets.Dropdown(options=['Manual', 'Automatic'], description='Transmission:')
owner_widget = widgets.Dropdown(options=['First', 'Second', 'Third', 'Fourth & Above'], description='Owner:')

# Display widgets
display(year_widget, kms_widget, fuel_widget, seller_widget, transmission_widget, owner_widget)

# Button to predict
button = widgets.Button(description="Predict Price")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    # Capture user input
    year = year_widget.value
    kms_driven = kms_widget.value
    fuel_type = fuel_widget.value
    seller_type = seller_widget.value
    transmission = transmission_widget.value
    owner = owner_widget.value

    # Create DataFrame
    user_data = {
        'Year': [year],
        'Kms_Driven': [kms_driven],
        'Fuel_Type': [fuel_type],
        'Seller_Type': [seller_type],
        'Transmission': [transmission],
        'Owner': [owner]
    }
    user_df = pd.DataFrame(user_data)

    # One-hot encode
    user_df_encoded = pd.get_dummies(user_df)

    # Align with training columns
    user_df_encoded = user_df_encoded.reindex(columns=feature_columns, fill_value=0)

    # Scale
    user_scaled = scaler.transform(user_df_encoded)

    # Predict
    predicted_price = model.predict(user_scaled)

    # Display result
    with output:
        output.clear_output()
        print("\n✅ Predicted Car Price: ₹{:.2f} Lakhs".format(predicted_price[0]))

# Connect button to prediction function
button.on_click(on_button_clicked)


IntSlider(value=2015, description='Year:', max=2024, min=2000)

IntText(value=50000, description='KMs Driven:')

Dropdown(description='Fuel Type:', options=('Petrol', 'Diesel', 'CNG', 'Electric'), value='Petrol')

Dropdown(description='Seller Type:', options=('Dealer', 'Individual'), value='Dealer')

Dropdown(description='Transmission:', options=('Manual', 'Automatic'), value='Manual')

Dropdown(description='Owner:', options=('First', 'Second', 'Third', 'Fourth & Above'), value='First')

Button(description='Predict Price', style=ButtonStyle())

Output()